In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.trend import sma_indicator, ema_indicator, macd_diff
from ta.momentum import rsi
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor
from sklearn.feature_selection import GenericUnivariateSelect, f_regression
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader



In [2]:
# === 1. Загрузка данных по акции ===
ticker = "AAPL"  # можно заменить на MSFT, TSLA и т.д.
data = yf.download(ticker, start="2020-01-01", end="2025-10-01")

C:\Users\Данил\AppData\Local\Temp\ipykernel_19100\3037679771.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2025-10-01")
[*********************100%***********************]  1 of 1 completed


In [2]:
# === 1. Загрузка данных по акции ===
ticker = "TSLA"  # можно заменить на MSFT, TSLA и т.д.
data = yf.download(ticker, start="2020-01-01", end="2025-10-01")

C:\Users\Данил\AppData\Local\Temp\ipykernel_19772\3492761586.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2025-10-01")
[*********************100%***********************]  1 of 1 completed


In [110]:
# === 1. Загрузка данных по акции ===
ticker = "MSFT"  # можно заменить на MSFT, TSLA и т.д.
data = yf.download(ticker, start="2020-01-01", end="2025-11-10")

C:\Users\Данил\AppData\Local\Temp\ipykernel_13496\1425222090.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end="2025-11-10")
[*********************100%***********************]  1 of 1 completed


In [3]:
len(data)

1444

In [4]:
def normalize_yf_dataframe(df):
    df = df.copy()
    
    if isinstance(df.columns, pd.MultiIndex):
        df.columns.names = ['DataType', 'Ticker']
        df = df.stack(level=1).reset_index()
        df = df.rename(columns={'level_1': 'Ticker'})
    else:
        df = df.reset_index()
        df['Ticker'] = df.columns[1].split()[0] if ' ' in df.columns[1] else 'UNKNOWN'

    df = df.rename(columns=str.capitalize)
    cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = df[[c for c in cols if c in df.columns]]

    return df

data = normalize_yf_dataframe(data)

C:\Users\Данил\AppData\Local\Temp\ipykernel_19772\3829911924.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()


In [5]:

data.to_csv('MSFT.csv')

In [6]:
df = pd.read_csv('MSFT.csv')

df

,Unnamed: 0,Date,Ticker,Open,High,Low,Close,Volume
0,0,2020-01-02,TSLA,28.299999,28.713333,28.114000,28.684000,142981500
1,1,2020-01-03,TSLA,29.366667,30.266666,29.128000,29.534000,266677500
2,2,2020-01-06,TSLA,29.364668,30.104000,29.333332,30.102667,151995000
3,3,2020-01-07,TSLA,30.760000,31.441999,30.224001,31.270666,268231500
4,4,2020-01-08,TSLA,31.580000,33.232666,31.215334,32.809334,467164500
...,...,...,...,...,...,...,...,...
1439,1439,2025-09-24,TSLA,429.829987,444.209991,429.029999,442.790009,93133600
1440,1440,2025-09-25,TSLA,435.239990,435.350006,419.079987,423.390015,96746400
1441,1441,2025-09-26,TSLA,428.299988,440.470001,421.019989,440.399994,101628200
1442,1442,2025-09-29,TSLA,444.350006,450.980011,439.500000,443.209991,79491500


In [7]:
data = df

In [8]:
data['Date'] = pd.to_datetime(data['Date'])

In [9]:
data.isna().sum()

Unnamed: 0    0
Date          0
Ticker        0
Open          0
High          0
Low           0
Close         0
Volume        0
dtype: int64

In [10]:
def make_features(data)->pd.DataFrame:
    data["SMA_10"] = sma_indicator(data["Close"], window=10)
    data["EMA_10"] = ema_indicator(data["Close"], window=10)
    data["RSI_14"] = rsi(data["Close"], window=14)
    data["MACD"] = macd_diff(data["Close"])


    data["DayOfWeek"] = data['Date'].dt.dayofweek
    data["Month"] = data['Date'].dt.month

    data["Next_Close"] = data["Close"].shift(-1)

    data = data.dropna()

    return data

data = make_features(data)

In [248]:
data.dropna(axis=1, inplace=True)

In [565]:
data["SMA_10"] = sma_indicator(data["Close"], window=10)
data["EMA_10"] = ema_indicator(data["Close"], window=10)
data["RSI_14"] = rsi(data["Close"], window=14)
data["MACD"] = macd_diff(data["Close"])


data["DayOfWeek"] = data['Date'].dt.dayofweek
data["Month"] = data['Date'].dt.month

data["Next_Close"] = data["Close"].shift(-1)

data = data.dropna()
# data.to_csv('AAPL.csv')




In [11]:
features = ["Open", "High", "Low", "Close",  "SMA_10", "EMA_10", "RSI_14", "MACD", "DayOfWeek", "Month"]

In [119]:
features = ["Open", "High", "Low", "Close"]

In [66]:
data['Label'] = np.where(
    (data['Next_Close'] - data['Close']) / data['Close'] >= 0.005,  # рост >= 1%
    1,
    0
)
    

C:\Users\Данил\AppData\Local\Temp\ipykernel_13496\1282796800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Label'] = np.where(


In [67]:
data['Label'].value_counts()

Label
0    860
1    550
Name: count, dtype: int64

In [12]:
def make_sliding_window(data, feature_cols, target_col, window_size=5):
    X, y = [], []
    values = data[feature_cols].values
    targets = data[target_col].values

    for i in range(window_size, len(data)):
        X.append(values[i - window_size:i])
        y.append(targets[i])
    
    return np.array(X), np.array(y)

In [146]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()

In [13]:

X, y = make_sliding_window(data, features, 'Next_Close', 10)
X = X.reshape(X.shape[0], -1)  
# X=scaler_x.fit_transform(X)
# y = y.reshape(-1, 1)             
# y=scaler_y.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = XGBRegressor()
model.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [14]:
def pip(model, df, window_size, features, target, use_scaler: bool = False):
    X, y = make_sliding_window(df, features, target, window_size)
    X = X.reshape(X.shape[0], -1)

    scaler_x = None
    scaler_y = None

    if use_scaler:
        scaler_x = StandardScaler()
        X = scaler_x.fit_transform(X)

        if not np.issubdtype(y.dtype, np.integer):
            scaler_y = StandardScaler()
            y = scaler_y.fit_transform(y.reshape(-1, 1)).ravel()

    split = int(len(X) * 0.7)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    if use_scaler and scaler_y is not None:
        y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).ravel()
        y_test = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()

    if np.issubdtype(type(y_train[0]), np.integer):
        y_pred_class = np.round(y_pred).astype(int)
        print(classification_report(y_test, y_pred_class))

    else:
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        print("Regression Metrics:")
        print(f"MSE:  {mse:.4f}")
        print(f"RMSE: {rmse:.4f}")
        print(f"R²:   {r2:.4f}")

    return model




In [53]:
pip(
    model=LinearRegression(),
    df=data,
    window_size= 2,
    features=features,
    target="Next_Close",
    use_scaler=False
)


Regression Metrics:
MSE:  272.8132
RMSE: 16.5171
R²:   0.9584


LinearRegression()

In [ ]:
X = data[features]
y = data['Label']

X, y = make_sliding_window(data, features, 'Label', 2)
X = X.reshape(X.shape[0], -1)  
X_train, X_test, y_train, y_test = X[:int(len(X)*0.7)], X[int(len(X)*0.7):], y[:int(len(X)*0.7)], y[int(len(X)*0.7):]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = LogisticRegression(penalty='l2')
model.fit(X_train, y_train)

c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [35]:
X = data[features]
y = data['Next_Close']
X_train, X_test, y_train, y_test = X[:int(len(X)*0.7)], X[int(len(X)*0.7):], y[:int(len(X)*0.7)], y[int(len(X)*0.7):]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [653]:

X, y = make_sliding_window(data, features, 'Label', 20)
X = X.reshape(X.shape[0], -1)  

# X=scaler_x.fit_transform(X)
X_train, X_test, y_train, y_test = X[:int(len(X)*0.9)], X[int(len(X)*0.9):], y[:int(len(X)*0.9)], y[int(len(X)*0.9):]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1
)
# model = RandomForestClassifier()
model.fit(X_train, y_train)


c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:23:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [291]:
import numpy as np

split = int(len(data) * 0.7)
df_train = data.iloc[:split]
df_test  = data.iloc[split:]

def simulate_trading(model, X_test, prices, threshold=0.5,
                     initial_balance=10000, position_size=1.0):
    """
    model          — обученная модель (должна иметь predict_proba или predict)
    X_test         — признаки
    prices         — массив цен (Close)
    threshold      — порог вероятности для покупки
    initial_balance — стартовый капитал
    position_size  — доля капитала в одной сделке (0.0 — 1.0)
    """

    balance = initial_balance
    position = 0      # 0 = нет позиции, 1 = куплено
    entry_price = 0.0

    portfolio_values = []
    actions = []      # BUY / SELL / HOLD

    # --- выбираем способ предсказания ---
    if hasattr(model, "predict_proba"):
        preds = model.predict_proba(X_test)[:, 1]
    else:
        preds = model.predict(X_test)
        if preds.ndim > 1:
            preds = preds[:, 0]

    for i in range(len(preds)):

        price = prices[i]
        prob = preds[i]
        action = "HOLD"

        # --- BUY ---
        if prob >= threshold and position == 0:
            position = position_size
            entry_price = price
            balance -= price * position
            action = "BUY"

        # --- SELL ---
        elif prob < threshold and position > 0:
            balance += price * position
            position = 0
            action = "SELL"

        # --- Track portfolio value ---
        portfolio_value = balance + position * price
        portfolio_values.append(portfolio_value)
        actions.append(action)

    # --- If holding position at the end, sell it ---
    if position > 0:
        balance += prices[-1] * position
        position = 0

    final_balance = balance
    total_return = (final_balance - initial_balance) / initial_balance

    # --- Drawdown ---
    pv = np.array(portfolio_values)
    running_max = np.maximum.accumulate(pv)
    max_drawdown = np.min((pv - running_max) / running_max)

    # --- Simple Sharpe ---
    if len(pv) > 1:
        daily_returns = np.diff(pv) / pv[:-1]
        sharpe = np.mean(daily_returns) / (np.std(daily_returns) + 1e-8)
    else:
        sharpe = 0.0

    metrics = {
        "final_balance": final_balance,
        "total_return": total_return,
        "sharpe": sharpe,
        "max_drawdown": max_drawdown,
        "actions": actions,
        "portfolio_values": portfolio_values
    }

    return metrics
metrics = simulate_trading(
    model,
    X_test,
    prices=df_test["Close"].values,
    threshold=0.29,         # выше — реже сделки, ниже — больше сделок
    initial_balance=10000,
    position_size=1.0       # покупаем на весь депозит
)

print("RESULTS:", metrics)

RESULTS: {'final_balance': np.float64(10012.81460571289), 'total_return': np.float64(0.0012814605712890626), 'sharpe': np.float64(0.01666683036422692), 'max_drawdown': np.float64(-0.0030644187615490354), 'actions': ['HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'SELL', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'SELL', 'HOLD', 'BUY', 'SELL', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'SELL', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'SELL', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'HOLD', 'SELL', 'BUY', 'HOLD', 'SELL', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'BUY', 'SELL', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 'HOLD', 

In [254]:
len(X_train)

1264

In [81]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Regression Metrics:")
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")

Regression Metrics:
MSE:  15.2851
RMSE: 3.9096
R²:   0.9716


In [80]:

y_pred = model.predict(X_test)
mae = model.score(X_test,y_test)

mae


0.9716294583658754

In [573]:
def func(X_train, X_test, y_train, y_test, w):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    x_train=list(X_train)
    y_train_ = list(y_train)


    y_pred = []

    for i in range(0, len(X_test), w):
        batch_x = X_test[i:i+w]
        batch_y = y_test[i:i+w]
        x_train.extend(batch_x)
        y_train_.extend(batch_y)
        y_batch_pred = model.predict(batch_x)
        y_pred.extend(y_batch_pred)

        # model.save_model("tmp.json")


        model.fit(
            x_train, 
            y_train_
        )


    print(classification_report(y_test, np.array(y_pred)))

    
func(X_train, X_test, y_train, y_test, 20)


              precision    recall  f1-score   support

           0       0.58      0.84      0.68       158
           1       0.47      0.18      0.26       120

    accuracy                           0.56       278
   macro avg       0.52      0.51      0.47       278
weighted avg       0.53      0.56      0.50       278



In [644]:
import xgboost as xgb
from sklearn.metrics import classification_report

def online_xgb(X_train, y_train, X_test, y_test, batch):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "max_depth": 5,
        "eta": 0.1
    }
    booster = xgb.train(params, dtrain, num_boost_round=50)

    y_pred = []

    for i in range(0, len(X_test), batch):
        bx = X_test[i:i+batch]
        by = y_test[i:i+batch]

        dpred = xgb.DMatrix(bx)
        preds = (booster.predict(dpred) > 0.5).astype(int)
        y_pred.extend(preds)

        btrain = xgb.DMatrix(bx, label=by)
        booster = xgb.train(
            params,
            btrain,
            num_boost_round=5,
            xgb_model=booster  
        )

    print(classification_report(y_test, y_pred))
    
    
online_xgb(X_train, y_train, X_test, y_test, batch=30)


              precision    recall  f1-score   support

           0       0.60      0.65      0.62       158
           1       0.48      0.43      0.45       119

    accuracy                           0.55       277
   macro avg       0.54      0.54      0.54       277
weighted avg       0.55      0.55      0.55       277



In [ ]:
from sklearn.linear_model import SGDClassifier

def online_logreg(X_train, y_train, X_test, y_test, batch=20):

    # SGD = Logistic Regression with online learning
    model = SGDClassifier(
        loss="log_loss",       # логистическая регрессия
        learning_rate="optimal",
        max_iter=1,
        warm_start=True        # позволяет сохранять веса
    )

    # Инициализация классов (partial_fit требует список)
    model.partial_fit(X_train[:batch], y_train[:batch], classes=np.array([0, 1]))

    y_pred = []

    for i in range(0, len(X_test), batch):
        bx = X_test[i:i + batch]
        by = y_test[i:i + batch]

        # ✅ Предсказания
        preds = (model.predict_proba(bx)[:, 1] > 0.63).astype(int)
        y_pred.extend(preds)

        # ✅ Дообучение на текущем батче
        model.partial_fit(bx, by)

    print(classification_report(y_test, y_pred))
online_logreg(X_train, y_train, X_test, y_test, batch=10)

In [17]:
class StockDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [18]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)

X_train: (1120, 100)
y_train: (1120,)


In [19]:
from torch.utils.data import WeightedRandomSampler

X, y = make_sliding_window(data, features, 'Next_Close', 20)

split = int(len(X) * 0.7)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# class_counts = np.bincount(y_train.astype(int))
# weights = 1. / class_counts
# sample_weights = [weights[int(t)] for t in y_train]
# sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(StockDataset(X_train, y_train), batch_size=32)
test_loader = DataLoader(StockDataset(X_test, y_test), batch_size=32, shuffle=False)

In [ ]:
train_loader = DataLoader(
    StockDataset(X_train.values, y_train.values),
    batch_size=32,
    shuffle=True
)
test_loader = DataLoader(
    StockDataset(X_test.values, y_test.values),
    batch_size=32,
    shuffle=False
)

In [385]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # берем выход последнего шага
        out = self.fc(out)
        return self.sigmoid(out)

In [133]:
import torch
import torch.nn as nn

class CNN_LSTM_Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, cnn_out=32, kernel_size=3):
        super().__init__()
        
        # --- 1. CNN слой для извлечения фич ---
        # x: (batch, seq_len, features) → нужно (batch, features, seq_len)
        self.conv1 = nn.Conv1d(
            in_channels=input_size,
            out_channels=cnn_out,
            kernel_size=kernel_size,
            padding=kernel_size // 2
        )
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(cnn_out)

        # --- 2. LSTM после CNN ---
        self.lstm = nn.LSTM(
            input_size=cnn_out,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )

        # --- 3. Линейный классификатор ---
        self.fc = nn.Linear(hidden_size, 1)

        # --- 4. Сигмоида для бинарной классификации ---
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x shape: (batch, seq_len, input_size)

        # ---- CNN expects (batch, input_channels, seq_len) ----
        x = x.permute(0, 2, 1)  # → (batch, features, seq_len)

        # --- CNN ---
        x = self.conv1(x)
        x = self.relu(x)
        x = self.bn(x)

        # вернуть обратно в формат для LSTM:
        # (batch, seq_len, cnn_out)
        x = x.permute(0, 2, 1)

        # --- LSTM ---
        out, _ = self.lstm(x)

        # берём последний timestep
        out = out[:, -1, :]  # (batch, hidden_size)

        # --- классификация ---
        out = self.fc(out)
        return self.sigmoid(out)


In [20]:
import torch
import torch.nn as nn

# --- Attention, совместимый с LSTM ---
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_out):
        # lstm_out: (batch, seq_len, hidden_dim)
        weights = torch.softmax(self.attn(lstm_out), dim=1)
        context = torch.sum(weights * lstm_out, dim=1)
        return context


class CNN_BiLSTM_Attention(nn.Module):
    def __init__(self, input_size, hidden_size, cnn_out=32, kernel_size=3, dropout=0.3):
        super().__init__()

        # --- CNN ---
        self.conv = nn.Conv1d(
            in_channels=input_size,
            out_channels=cnn_out,
            kernel_size=kernel_size,
            padding=kernel_size//2
        )
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(cnn_out)

        # --- BiLSTM ---
        self.lstm = nn.LSTM(
            input_size=cnn_out,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
            dropout=dropout
        )

        # --- Attention ---
        self.attention = Attention(hidden_dim=hidden_size * 2)

        # --- Classifier ---
        self.fc = nn.Linear(hidden_size * 2, 1)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch, seq_len, features)
        x = x.permute(0, 2, 1)     # → (batch, features, seq_len)
        x = self.relu(self.bn(self.conv(x)))
        x = x.permute(0, 2, 1)     # → (batch, seq_len, cnn_out)

        lstm_out, _ = self.lstm(x)
        lstm_out = self.dropout(lstm_out)

        context = self.attention(lstm_out)  # (batch, hidden_dim*2)

        output = self.fc(context)
        return output   


In [33]:
def train(model, loader, optimizer, loss_fn, device, epochs=30):
    model.train()
    for epoch in range(epochs):
        losses = []

        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)

            optimizer.zero_grad()
            pred = model(Xb).flatten()

            loss = loss_fn(pred, yb)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        print(f"Epoch {epoch+1}/{epochs} | Train Loss={np.mean(losses):.4f}")






# ============================================================
# 9) ОЦЕНКА
# ============================================================
def evaluate(model, X, y_true_scaled):
    model.eval()
    X = torch.tensor(X, dtype=torch.float32).to(device)

    with torch.no_grad():
        preds = model(X).cpu().numpy().flatten()

    # вернуть масштаб
    preds_real = scaler_y.inverse_transform(preds.reshape(-1, 1)).flatten()
    y_real = scaler_y.inverse_transform(y_true_scaled.reshape(-1, 1)).flatten()

    mae = mean_absolute_error(y_real, preds_real)
    rmse = mean_squared_error(y_real, preds_real) ** 0.5
    mape = np.mean(np.abs((y_real - preds_real) / y_real)) * 100
    r2 = r2_score(y_real, preds_real)

    print("\n=== METRICS ===")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²:   {r2:.4f}")

    return preds_real, y_real

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# model = CNN_BiLSTM_Attention(input_size=X.shape[2], hidden_size=64).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train(model, train_loader, epochs=40, optimizer=optimizer, loss_fn=loss_fn, device=device)


Using device: cuda
Epoch 1/40 | Train Loss=2684.3537
Epoch 2/40 | Train Loss=2705.4990
Epoch 3/40 | Train Loss=2707.5581
Epoch 4/40 | Train Loss=2688.0476
Epoch 5/40 | Train Loss=2651.5642
Epoch 6/40 | Train Loss=2650.1644
Epoch 7/40 | Train Loss=2659.5071
Epoch 8/40 | Train Loss=2634.6533
Epoch 9/40 | Train Loss=2583.9802
Epoch 10/40 | Train Loss=2605.1590
Epoch 11/40 | Train Loss=2546.5889
Epoch 12/40 | Train Loss=2587.1729
Epoch 13/40 | Train Loss=2557.1984
Epoch 14/40 | Train Loss=2498.6839
Epoch 15/40 | Train Loss=2518.4725
Epoch 16/40 | Train Loss=2462.5995
Epoch 17/40 | Train Loss=2396.1569
Epoch 18/40 | Train Loss=2312.5571
Epoch 19/40 | Train Loss=2317.6472
Epoch 20/40 | Train Loss=2257.7385
Epoch 21/40 | Train Loss=2213.3434
Epoch 22/40 | Train Loss=2177.9985
Epoch 23/40 | Train Loss=2138.0938
Epoch 24/40 | Train Loss=2148.7209
Epoch 25/40 | Train Loss=2210.4094
Epoch 26/40 | Train Loss=2201.3467
Epoch 27/40 | Train Loss=2132.7107
Epoch 28/40 | Train Loss=2031.4596
Epoch 29/4

In [34]:
def evaluate(model, X, y_true):
    model.eval()
    X = torch.tensor(X, dtype=torch.float32).to(device)

    with torch.no_grad():
        preds = model(X).cpu().numpy().flatten()

    # Предполагается, что y_true и preds уже в "реальных" масштабах
    mae = mean_absolute_error(y_true, preds)
    rmse = mean_squared_error(y_true, preds)
    mape = np.mean(np.abs((y_true - preds) / y_true)) * 100
    r2 = r2_score(y_true, preds)

    print("\n=== METRICS ===")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²:   {r2:.4f}")

    return preds, y_true

preds, y_real = evaluate(model, X_test[:-1], y_test[:-1])


=== METRICS ===
MAE:  86.3805
RMSE: 12990.4051
MAPE: 27.35%
R²:   -0.9980


In [124]:

input_size = len(features)
# input_size = 70
hidden_size = 64
# model = LSTMClassifier(input_size, hidden_size)
# model= CNN_LSTM_Classifier(input_size, hidden_size)
model = CNN_BiLSTM_Attention(
    input_size,    # например 5
    hidden_size=64,
    cnn_out=32,
    kernel_size=3
)

pos_weight = torch.tensor([len(y_train[y_train==0]) / len(y_train[y_train==1])])

criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


ZeroDivisionError: division by zero

In [126]:
def prepare_data(X, y, train_ratio=0.7, val_ratio=0.15):
    n = len(X)
    train_end = int(n * train_ratio)
    val_end = int(n * (train_ratio + val_ratio))

    X_train, y_train = X[:train_end], y[:train_end]
    X_val, y_val = X[train_end:val_end], y[train_end:val_end]
    X_test, y_test = X[val_end:], y[val_end:]

    # масштабируем ТОЛЬКО признаки
    scaler = StandardScaler()
    X_train_scaled = X_train.reshape(-1, X_train.shape[-1])
    X_val_scaled = X_val.reshape(-1, X_val.shape[-1])
    X_test_scaled = X_test.reshape(-1, X_test.shape[-1])

    scaler.fit(X_train_scaled)

    X_train_scaled = scaler.transform(X_train_scaled).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val_scaled).reshape(X_val.shape)
    X_test_scaled = scaler.transform(X_test_scaled).reshape(X_test.shape)

    return (X_train_scaled, y_train,
            X_val_scaled, y_val,
            X_test_scaled, y_test)



In [128]:
def train_model(model, train_loader, val_loader, device, epochs=30, lr=1e-3):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)

            pred = model(Xb)
            loss = criterion(pred, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                pred = model(Xb)
                loss = criterion(pred, yb)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

def evaluate_regression(model, X_test, y_test, device):
    model.eval()
    with torch.no_grad():
        preds = model(torch.tensor(X_test, dtype=torch.float32).to(device))
        preds = preds.cpu().numpy().flatten()

    y_true = y_test.flatten()

    mae = mean_absolute_error(y_true, preds)
    rmse = mean_squared_error(y_true, preds)
    r2 = r2_score(y_true, preds)
    mape = np.mean(np.abs((y_true - preds) / y_true)) * 100

    print("\n===== TEST METRICS =====")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²:   {r2:.4f}")

    return preds


In [135]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# X.shape == (samples, seq_len, features)
# y.shape == (samples,)
X_train, y_train, X_val, y_val, X_test, y_test = prepare_data(X, y)

train_loader = DataLoader(StockDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(StockDataset(X_val, y_val), batch_size=32, shuffle=False)

# model = CNN_BiLSTM_Attention(
#     input_size=X.shape[2],
#     hidden_size=64
# ).to(device)
# model = model.to(device)
train_model(model, train_loader, val_loader, device, epochs=30, lr=1e-3)

preds = evaluate_regression(model, X_test, y_test, device)

Using device: cuda


c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/30 | Train Loss: 100233.5955 | Val Loss: 234519.8691
Epoch 2/30 | Train Loss: 98984.5310 | Val Loss: 239940.7461
Epoch 3/30 | Train Loss: 99467.5513 | Val Loss: 226468.8340
Epoch 4/30 | Train Loss: 98929.7828 | Val Loss: 226314.7891
Epoch 5/30 | Train Loss: 98019.1647 | Val Loss: 221419.6914
Epoch 6/30 | Train Loss: 99258.2759 | Val Loss: 230187.7598
Epoch 7/30 | Train Loss: 99606.1843 | Val Loss: 241588.8164
Epoch 8/30 | Train Loss: 99832.6294 | Val Loss: 231439.5977
Epoch 9/30 | Train Loss: 98076.6323 | Val Loss: 241077.5234
Epoch 10/30 | Train Loss: 98512.1714 | Val Loss: 236915.6660
Epoch 11/30 | Train Loss: 99117.8936 | Val Loss: 240716.3809
Epoch 12/30 | Train Loss: 97909.9526 | Val Loss: 237647.0684
Epoch 13/30 | Train Loss: 98116.4734 | Val Loss: 236903.5508
Epoch 14/30 | Train Loss: 98506.6494 | Val Loss: 223523.6387
Epoch 15/30 | Train Loss: 96829.8242 | Val Loss: 237922.3711
Epoch 16/30 | Train Loss: 99756.1721 | Val Loss: 224113.7168
Epoch 17/30 | Train Loss: 98573.

In [626]:
EPOCHS = 20
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss = {total_loss/len(train_loader):.4f}")

# ======================
# 7️⃣ Оценка на тесте
# ======================
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch).squeeze()
        preds = (preds > 0.5).int()
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print(f"\nТочность модели: {accuracy*100:.2f}%")

Epoch 1/20, Loss = 0.6901
Epoch 2/20, Loss = 0.6856
Epoch 3/20, Loss = 0.6873
Epoch 4/20, Loss = 0.6853
Epoch 5/20, Loss = 0.6843
Epoch 6/20, Loss = 0.6867
Epoch 7/20, Loss = 0.6876
Epoch 8/20, Loss = 0.6876
Epoch 9/20, Loss = 0.6857
Epoch 10/20, Loss = 0.6870
Epoch 11/20, Loss = 0.6860
Epoch 12/20, Loss = 0.6859
Epoch 13/20, Loss = 0.6869
Epoch 14/20, Loss = 0.6872
Epoch 15/20, Loss = 0.6853
Epoch 16/20, Loss = 0.6855
Epoch 17/20, Loss = 0.6853
Epoch 18/20, Loss = 0.6852
Epoch 19/20, Loss = 0.6853
Epoch 20/20, Loss = 0.6856

Точность модели: 57.04%


In [363]:
n_steps = 5     # длина окна (дней)
n_features = len(features)  # число признаков
hidden_size = 64
batch_size = 32
epochs = 20
class AttentionModel(nn.Module):
    def __init__(self, n_features, hidden_size, n_heads=4, n_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(n_features, hidden_size)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=n_heads,
            dim_feedforward=hidden_size * 2,
            dropout=0.1,
            activation='relu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (batch, seq_len, features)
        x = self.input_proj(x)
        x = self.transformer(x)
        out = x.mean(dim=1)  # усредняем по времени
        out = self.fc(out)
        return self.sigmoid(out)

# --- Инициализация модели ---
model = AttentionModel(n_features, hidden_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# --- Обучение ---
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# --- Тест ---
# ======================
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch).squeeze()
        preds = (preds > 0.5).int()
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print(f"\nТочность модели: {accuracy*100:.2f}%")


Epoch 1/20, Loss: 0.6880
Epoch 2/20, Loss: 0.6905
Epoch 3/20, Loss: 0.6935
Epoch 4/20, Loss: 0.6881
Epoch 5/20, Loss: 0.6909
Epoch 6/20, Loss: 0.6890
Epoch 7/20, Loss: 0.6847
Epoch 8/20, Loss: 0.6882
Epoch 9/20, Loss: 0.6869
Epoch 10/20, Loss: 0.6894
Epoch 11/20, Loss: 0.6897
Epoch 12/20, Loss: 0.6891
Epoch 13/20, Loss: 0.6845
Epoch 14/20, Loss: 0.6840
Epoch 15/20, Loss: 0.6846
Epoch 16/20, Loss: 0.6867
Epoch 17/20, Loss: 0.6847
Epoch 18/20, Loss: 0.6878
Epoch 19/20, Loss: 0.6877
Epoch 20/20, Loss: 0.6868

Точность модели: 53.60%


In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# --- Перевод модели в режим оценки ---
model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in train_loader:
        preds = model(X_batch).squeeze()
        preds = torch.round(preds)  # округляем вероятности к 0/1
        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# --- Вывод отчёта ---
print("📈 Classification Report:")
print(classification_report(y_true, y_pred, digits=4))

# --- Матрица ошибок ---
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


In [305]:
epochs = 20
model.train()

for epoch in range(epochs):
    total_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        outputs = model(X_batch).squeeze()  # logits
        loss = criterion(outputs, y_batch.float())

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/20, Loss: 0.7959
Epoch 2/20, Loss: 0.7985
Epoch 3/20, Loss: 0.7959
Epoch 4/20, Loss: 0.7930
Epoch 5/20, Loss: 0.7919
Epoch 6/20, Loss: 0.7930
Epoch 7/20, Loss: 0.7937
Epoch 8/20, Loss: 0.7960
Epoch 9/20, Loss: 0.7895
Epoch 10/20, Loss: 0.7940
Epoch 11/20, Loss: 0.7890
Epoch 12/20, Loss: 0.7945
Epoch 13/20, Loss: 0.7873
Epoch 14/20, Loss: 0.7881
Epoch 15/20, Loss: 0.7860
Epoch 16/20, Loss: 0.7826
Epoch 17/20, Loss: 0.7903
Epoch 18/20, Loss: 0.7852
Epoch 19/20, Loss: 0.7882
Epoch 20/20, Loss: 0.7837


In [639]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        logits = model(X_batch).squeeze()
        probs = torch.sigmoid(logits)

        # динамический порог (можно менять 0.55–0.65)
        preds = (probs > 0.607).int()

        y_true.extend(y_batch.numpy())
        y_pred.extend(preds.numpy())

from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))


              precision    recall  f1-score   support

         0.0       0.60      0.54      0.57       158
         1.0       0.46      0.52      0.49       119

    accuracy                           0.53       277
   macro avg       0.53      0.53      0.53       277
weighted avg       0.54      0.53      0.54       277

[[86 72]
 [57 62]]


In [494]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [519]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

# X_train_flat = X_train.reshape(X_train_nn.shape[0], -1)  # (1112, 200)
# X_test_flat = X_test.reshape(X_test_nn.shape[0], -1)

# --- Масштабируем для sklearn моделей ---
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# --- Классические модели ---
logreg = LogisticRegression(max_iter=1000)
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
xgb_clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False,
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1
)

# --- Обёртка для нейросети ---
class TorchNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model, device="cpu"):
        self.model = model.to(device)  # переносим модель на нужное устройство
        self.device = device

    def fit(self, X, y):
        # модель уже обучена
        return self

    def predict(self, X):
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)  # данные на том же устройстве
        with torch.no_grad():
            outputs = self.model(X_tensor)
            if outputs.shape[1] == 1:
                preds = (torch.sigmoid(outputs) > 0.1).cpu().numpy().astype(int).flatten()
            else:
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
        return preds

nn_clf = TorchNNClassifier(model=model, device="cuda")

# --- Ensemble ---
voting_clf = VotingClassifier(
    estimators=[
        ('lr', logreg),
        ('rf', rf_clf),
        ('xgb', xgb_clf),
        # ('nn', nn_clf)  
    ],
    voting='hard'
)

# --- Обучение ensemble ---
# Для sklearn моделей используем X_train_scaled, для nn — X_train_nn
# Можно обучать через sklearn wrapper с разными входами, либо отдельно: 
# sklearn модели на X_train_scaled, nn на X_train_nn

# Простое решение: обучаем классические модели отдельно
logreg.fit(X_train_scaled, y_train)
rf_clf.fit(X_train_scaled, y_train)
xgb_clf.fit(X_train_scaled, y_train)
nn_clf.fit(X_train, y_train)  # нейросеть уже обучена

# --- Предсказания ---
# Классические модели
pred_lr = logreg.predict(X_test_scaled)
pred_rf = rf_clf.predict(X_test_scaled)
pred_xgb = xgb_clf.predict(X_test_scaled)
pred_nn = nn_clf.predict(X_test)

# --- Hard voting ---
preds_stack = np.vstack([pred_lr, pred_rf, pred_xgb, pred_nn]).T
y_pred = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=preds_stack)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

c:\Users\Данил\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:36:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.47      0.66      0.55       127
           1       0.56      0.37      0.44       147

    accuracy                           0.50       274
   macro avg       0.52      0.51      0.50       274
weighted avg       0.52      0.50      0.49       274

